In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import psycopg2
import time

In [7]:
service = Service(r'C:\Users\Administrator\OneDrive\Documents\Projects\Data Analysis Bootcamp by Lux\JobListings\chromedriver-win64\chromedriver.exe')
driver = webdriver.Chrome(service=service)

In [8]:
job_titles = ["Data Analyst","Data Engineer", "Data Scientist", "Python Developer","Machine Learning Engineer"]
base_url = "https://www.indeed.com/jobs?q=&l=Remote&from=searchOnHP&vjk=8e45d181912e40ec"

In [11]:
def scrape_jobs(job_title):
    jobs = []
    url = base_url.format(job_title.replace('','+'))
    driver.get(url)
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source,'html.parser')
    job_cards = soup.find_all('div', class_ = 'jobsearch-SerpJobCard')

    for job_card in job_cards:
        job = {}
        job['title'] = job_card.find('h2', class_ = 'title').text.strip()
        job['company'] = job_card.find('span', class_ = 'company').text.strip()
        job['location'] = job_card.find('div', class_ = 'recJobLoc').get('data-rc-loc')
        job['summary'] = job_card.find('div', class_ = 'summary').text.strip()
        job['salary'] = job_card.find('span', class_ = 'salaryText').text.strip() if job_card.find('span', class_='salaryText') else 'N/A'
        job['link'] = 'https://www.indeed.com' + job_card.find('a').get('href')
        jobs.append(job)
    return jobs
all_jobs = []
for title in job_titles:
    all_jobs.extend(scrape_jobs(title))

In [12]:
import psycopg2

In [14]:
conn = psycopg2.connect(
    dbname = "IndeedJobs",
    user = "postgres",
    password = "Gloria@22",
    host= "localhost",
    port = "5432"
)
cursor = conn.cursor()

In [15]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS job_listings (
    id SERIAL PRIMARY KEY,
    title TEXT,
    company TEXT,
    location TEXT,
    summary TEXT,
    salary TEXT,
    link TEXT
)
''')
conn.commit()


In [16]:
for job in all_jobs:
    cursor.execute('''
    INSERT INTO job_listings (title, company, location, summary, salary, link)
    VALUES (%s, %s, %s, %s, %s, %s)
    ''', (job['title'], job['company'], job['location'], job['summary'], job['salary'], job['link']))
conn.commit()
